# CamShift Object Tracking Algorithm

In [1]:
import cv2
import numpy as np

# code that initializes the webcam
cap = cv2.VideoCapture(0)

# code that takes the first frame of the video
ret, frame = cap.read()

# setups the default location of the window
r, h, c, w = 240, 100, 400, 160
track_window = (c, r, w, h)

# crops the region of interest for tracking
roi = frame[r:r+h, c:c+w]

# converts the cropped window to HSV color space
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Creates a mask between the lower and upper HSV bounds.
lower_purple = np.array([125, 0, 0])
upper_purple = np.array([175, 255, 255])
mask = cv2.inRange(hsv_roi, lower_purple, upper_purple)

# obtains the color Histogram of the ROI
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [100], [0, 180])

# code that normalizes values to lie between the range of 0, 255
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# code that setups the termination criteria
# it says that we stop calculating the centroid shift aftre ten iterations
# or if the centroid has moved at least 1 pixel.
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while True:
    # read webcam frame
    ret,frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    
    if ret == True:
        # converts to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # calc the histogram back projection
        # Each pixel value is its probability
        dst  = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        
        # code that applies the camShift algo to get the new location and also a new tracking window base on the object size.
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        
        # draws it on image
        # we use .polylines instead of .rectangles to rep Adaptive box based on the object size.
        # rather than just fixed rectangular boxes.
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame, [pts], True, 255, 2)
        
        cv2.imshow('CamShift Tracking', img2)
        
        if cv2.waitKey(1) == 13:
            break
    else:
        break
        
cap.release()
cv2.destroyAllWindows()